In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import random
import scipy
import os
import pickle

In [2]:
from preprocessing import inception_preprocessing

In [3]:
import inception_base

In [4]:
class Dogs_Cats_model(object):
    def __init__(self,learning_rate=1e-3,num_classes=2,initializer_scale=0.5):     
        self.learning_rate = learning_rate
        self.num_classes = num_classes
        self.initializer = tf.random_uniform_initializer(
                                            minval=-initializer_scale,
                                            maxval=initializer_scale)
        # A float32 Tensor with shape [batch_size, height, width, channels].
        self.images = None

        # A float32 scalar Tensor; the total loss for the trainer to optimize.
        self.final_loss = None

        # Global step Tensor.
        self.global_step = None
    def build_inputs_outputs(self):
        image_feed = tf.placeholder(dtype=tf.float32,shape=[None,299,299,3],name="image_feed")
        self.image_feed = image_feed
        self.ground_truth = tf.placeholder(tf.float32,[None,self.num_classes])
        
    def build_image_embedding(self):
        self.inception_output = inception_base.get_base_model(self.image_feed)
        self.fine_tune_input = tf.placeholder_with_default(self.inception_output,self.inception_output.get_shape(),name="fine_tune_input")
        self.inception_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="InceptionV4")
        with tf.variable_scope("fine_tune_layers") as scope:
            self.logits = tf.contrib.layers.fully_connected(inputs=self.fine_tune_input,
                                                          num_outputs=self.num_classes,
                                                          activation_fn = None,
                                                          weights_initializer=self.initializer,
                                                    
                                                          scope = scope)
    def build_loss(self):
        self.probabilities = tf.nn.softmax(self.logits,name="output_probs")
        with tf.name_scope("cross_entropy") as scope:
            loss = tf.nn.softmax_cross_entropy_with_logits(labels=self.ground_truth,logits=self.logits)
            with tf.name_scope('total'):
                  self.final_loss = tf.reduce_mean(loss)
    def build_train_step(self):
        with tf.name_scope("train") as scope:
            optimizer = tf.train.AdamOptimizer(self.learning_rate)
            self.train_step = optimizer.minimize(self.final_loss)
            
    def build_eval_step(self):
        with tf.name_scope('accuracy'):
            with tf.name_scope('correct_prediction'):
                prediction = tf.argmax(self.logits, 1)
                correct_prediction = tf.equal(prediction, tf.argmax(self.ground_truth, 1))
            with tf.name_scope('accuracy'):
                self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    def setup_global_step(self):
        """Sets up the global step Tensor."""
        global_step = tf.Variable(
            initial_value=0,
            name="global_step",
            trainable=False,
            collections=[tf.GraphKeys.GLOBAL_STEP, tf.GraphKeys.GLOBAL_VARIABLES])
        self.global_step = global_step
        
    def build(self):
        self.build_inputs_outputs()
        self.build_image_embedding()
        self.build_loss()
        self.build_train_step()
        self.build_eval_step()

## Getting input from file

In [5]:
# dict containing file_name and label
train_name_dict = {}

In [6]:
TRAIN_DIR_CAT = "../ignore/kaggle_cats_and_dogs/train/cat/"

In [7]:
TRAIN_DIR_DOG = "../ignore/kaggle_cats_and_dogs/train/dog/"

In [97]:
TEST_DIR = "../ignore/kaggle_cats_and_dogs/test/"

In [9]:
# list of np.float32 array containing the training images
train_data_unprocessed = []

# list of np.float32 array containing the test images
test_data_unprocessed = []

In [10]:
cats_files = [os.path.join(TRAIN_DIR_CAT,fl) for fl in os.listdir(TRAIN_DIR_CAT)]
dogs_files = [os.path.join(TRAIN_DIR_DOG,fl) for fl in os.listdir(TRAIN_DIR_DOG)]

In [98]:
test_files_names = os.listdir(TEST_DIR)

In [99]:
test_file_paths = [os.path.join(TEST_DIR,fl) for fl in test_files_names]

In [12]:
bs = 50


In [13]:
num_epochs = 100
out = []
out_labels =[]

## Making model

In [14]:
model = Dogs_Cats_model()

In [15]:
model.build()

In [16]:
sess = tf.Session()

In [17]:
sess.run(tf.global_variables_initializer())

In [18]:
saver = tf.train.Saver(var_list=model.inception_variables)
saver.restore(sess,"./inception_v4.ckpt")

INFO:tensorflow:Restoring parameters from ./inception_v4.ckpt


In [19]:
for start, end in zip(range(0, len(cats_files)+1, bs),range(49, len(cats_files)+1, bs)):
        cat_batch = [scipy.ndimage.imread(fl) for fl in cats_files[start:end]]
        dog_batch = [scipy.ndimage.imread(fl) for fl in dogs_files[start:end]]
        cat_batch_resized = [scipy.misc.imresize(cat,[299,299,3]) for cat in cat_batch]
        dog_batch_resized = [scipy.misc.imresize(dog,[299,299,3]) for dog in dog_batch]
        cat_batch_processed = [((2*(cat.astype(np.float32) / 255 ))-1) for cat in cat_batch_resized]
        dog_batch_processed = [((2*(dog.astype(np.float32) / 255 ))-1) for dog in dog_batch_resized]
        train_data_cat = [(cat,0) for cat in cat_batch_processed]
        train_data_dog = [(dog,1) for dog in dog_batch_processed]
        train_data_processed=[]
        for cat in train_data_cat:train_data_processed.append(cat) 
        for dog in train_data_dog:train_data_processed.append(dog)
        random.shuffle(train_data_processed)
        train_data = [t[0] for t in train_data_processed]
        train_labels = [t[1] for t in train_data_processed]
        train_labels_tuples = np.array([[1,0] if x==0 else [0,1] for x in train_labels])
        train_data_np = np.array(train_data)
        out.append(sess.run(model.inception_output,feed_dict={model.image_feed:train_data_np}))
        out_labels.append(train_labels_tuples)
        print("Processed ", start)

('Processed ', 0)
('Processed ', 50)
('Processed ', 100)
('Processed ', 150)
('Processed ', 200)
('Processed ', 250)
('Processed ', 300)
('Processed ', 350)
('Processed ', 400)
('Processed ', 450)
('Processed ', 500)
('Processed ', 550)
('Processed ', 600)
('Processed ', 650)
('Processed ', 700)
('Processed ', 750)
('Processed ', 800)
('Processed ', 850)
('Processed ', 900)
('Processed ', 950)
('Processed ', 1000)
('Processed ', 1050)
('Processed ', 1100)
('Processed ', 1150)
('Processed ', 1200)
('Processed ', 1250)
('Processed ', 1300)
('Processed ', 1350)
('Processed ', 1400)
('Processed ', 1450)
('Processed ', 1500)
('Processed ', 1550)
('Processed ', 1600)
('Processed ', 1650)
('Processed ', 1700)
('Processed ', 1750)
('Processed ', 1800)
('Processed ', 1850)
('Processed ', 1900)
('Processed ', 1950)
('Processed ', 2000)
('Processed ', 2050)
('Processed ', 2100)
('Processed ', 2150)
('Processed ', 2200)
('Processed ', 2250)
('Processed ', 2300)
('Processed ', 2350)
('Processed ', 

In [26]:
pickle.dump( out, open( "output_inception.p", "wb" ) )
pickle.dump( out_labels, open( "output_labels.p", "wb" ) )

In [27]:
np.save("./output_inception_numpy.npy",out)
np.save("./output_labels_numpy.npy",out_labels)

In [30]:
for tt in xrange(num_epochs):
    for i in range(len(out)):
        _,loss = sess.run([model.train_step,model.final_loss],feed_dict={model.fine_tune_input:out[i],model.ground_truth:out_labels[i]})
    print("Epoch- ", tt , "Loss = ",loss)

('Epoch- ', 0, 'Loss = ', 2.4434081e-05)
('Epoch- ', 1, 'Loss = ', 2.325998e-05)
('Epoch- ', 2, 'Loss = ', 2.2037253e-05)
('Epoch- ', 3, 'Loss = ', 2.0785295e-05)
('Epoch- ', 4, 'Loss = ', 1.9525956e-05)
('Epoch- ', 5, 'Loss = ', 1.8270144e-05)
('Epoch- ', 6, 'Loss = ', 1.7028793e-05)
('Epoch- ', 7, 'Loss = ', 1.5815254e-05)
('Epoch- ', 8, 'Loss = ', 1.4639259e-05)
('Epoch- ', 9, 'Loss = ', 1.3509318e-05)
('Epoch- ', 10, 'Loss = ', 1.2433952e-05)
('Epoch- ', 11, 'Loss = ', 1.1420463e-05)
('Epoch- ', 12, 'Loss = ', 1.0471298e-05)
('Epoch- ', 13, 'Loss = ', 9.5937703e-06)
('Epoch- ', 14, 'Loss = ', 8.785466e-06)
('Epoch- ', 15, 'Loss = ', 8.0403152e-06)
('Epoch- ', 16, 'Loss = ', 7.3522574e-06)
('Epoch- ', 17, 'Loss = ', 6.7188703e-06)
('Epoch- ', 18, 'Loss = ', 6.1280039e-06)
('Epoch- ', 19, 'Loss = ', 5.5760174e-06)
('Epoch- ', 20, 'Loss = ', 5.058052e-06)
('Epoch- ', 21, 'Loss = ', 4.5728962e-06)
('Epoch- ', 22, 'Loss = ', 4.1169046e-06)
('Epoch- ', 23, 'Loss = ', 3.6888632e-06)
('Epo

In [100]:
preds = {}

In [101]:
for start, end in zip(range(0, len(test_file_paths)-1,100),range(100, len(test_file_paths)+1,100)):
    test_batch = [scipy.ndimage.imread(fl) for fl in test_file_paths[start:end]]
    test_batch_resized = [scipy.misc.imresize(tst,[299,299,3]) for tst in test_batch]
    test_batch_processed = [((2*(tst.astype(np.float32) / 255 ))-1) for tst in test_batch_resized]
    train_data_np = np.array(test_batch_processed)
    out = sess.run(model.inception_output,feed_dict={model.image_feed:train_data_np})
    pred = sess.run(model.probabilities,feed_dict={model.fine_tune_input:out})
    pred_dog = [p[1] for p in pred]
    for i in range(start,end):
        preds[test_files_names[i]] = pred_dog[i-start]
    print("indices ", start , " end - ", end)
    

('indices ', 0, ' end - ', 100)
('indices ', 100, ' end - ', 200)
('indices ', 200, ' end - ', 300)
('indices ', 300, ' end - ', 400)
('indices ', 400, ' end - ', 500)
('indices ', 500, ' end - ', 600)
('indices ', 600, ' end - ', 700)
('indices ', 700, ' end - ', 800)
('indices ', 800, ' end - ', 900)
('indices ', 900, ' end - ', 1000)
('indices ', 1000, ' end - ', 1100)
('indices ', 1100, ' end - ', 1200)
('indices ', 1200, ' end - ', 1300)
('indices ', 1300, ' end - ', 1400)
('indices ', 1400, ' end - ', 1500)
('indices ', 1500, ' end - ', 1600)
('indices ', 1600, ' end - ', 1700)
('indices ', 1700, ' end - ', 1800)
('indices ', 1800, ' end - ', 1900)
('indices ', 1900, ' end - ', 2000)
('indices ', 2000, ' end - ', 2100)
('indices ', 2100, ' end - ', 2200)
('indices ', 2200, ' end - ', 2300)
('indices ', 2300, ' end - ', 2400)
('indices ', 2400, ' end - ', 2500)
('indices ', 2500, ' end - ', 2600)
('indices ', 2600, ' end - ', 2700)
('indices ', 2700, ' end - ', 2800)
('indices ', 

In [82]:
pred_clipped = [np.clip()]

In [86]:
preds['1.jpg']

1

In [105]:
with open("kaggle_dogs_cats_prediction.csv","w") as fl:
    fl.write("id,label\n")
    for i in range(12500):
        fl.write(str(i+1)+","+str(np.clip(preds[str(i+1)+".jpg"],0.03,0.97))+"\n")

In [91]:
x = [np.array([1,i]) for i in range(10)]

In [92]:
t = np.array(x)

In [95]:
v = [s[1] for s in t]

In [96]:
v

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]